In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
# Dense: to add layers(hidden, output)
from tensorflow.keras import Sequential
# Sequential: to add layers in sequence, to initialize ann model i.e. initially random weight will be assigned
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("C:/Users/gheva/OneDrive/Desktop/Alphabets_data.csv")
df

In [ ]:
df.isna().sum()

In [ ]:
sc = StandardScaler() # don't perform standardization on target column
x = df.drop(columns=['letter'])# exclude Exited column
y = df['letter']

In [ ]:
x = sc.fit_transform(x)
x

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.20, random_state=1)

In [ ]:
xtrain

In [ ]:
# create instance of Sequential class
model = Sequential()

#Add hidden layer, randomly 10 neurons are added
model.add(Dense(units=10,activation='relu'))

                #Add output layer
model.add(Dense(units=1, activation='sigmoid'))

                
#Establish the connection between the layers
model.compile(optimizer = 'adadelta',loss='binary_crossentropy',metrics=['accuracy'])

                #Fit the data, perform forward and back propagation
model.fit(xtrain,ytrain, epochs=100)

In [ ]:
xtest

In [ ]:
ypred = model.predict(xtest)
ypred

In [ ]:
ytest

In [ ]:
print(classification_report(ytest,ypred))

In [ ]:
ypred > 0.5

In [ ]:
ypred = ypred > 0.5
ypred

In [ ]:
ytest

In [ ]:

print(classification_report(ytest,ypred))

##Hyperparameter Tuning##

In [ ]:
!pip install -U keras-tuner

##Tuning the ANN Model##

In [ ]:
import keras_tuner as kt

In [ ]:
def optimizer_selection(hp):
    #create instance of sequential class
    model = Sequential()
    #Add hidden layer
    model.add(Dense(units=10, activation='relu'))
    #Add output layer
    model.add(Dense(units=1, activation='sigmoid'))
    #Optimizer selection
    optim = hp.Choice('optimizer', values = ['sgd','adam','rmsprop']) #value of optimizer is categorical so use choice()function else use hp.Int()
    model.compile(optimizer=optim, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
# similar to GridSearchCV() we will use here RandomSearch()
tuner = kt.RandomSearch(
    optimizer_selection,# model name-here Sequential() class is our model
    objective='val_accuracy', # increase accuracy of test data i.e. validation test accuracy or minimize 'val_loss'
    max_trials=3 # for each optimizer take 3 trials, can select 3 to 10
)

In [ ]:
tuner.search(xtrain,ytrain, epochs = 3, validation_data = (xtest,ytest))

In [ ]:

tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0] # get best model: rmsprop
model.fit(xtrain,ytrain, epochs = 100, validation_data = (xtest,ytest))

In [ ]:
model.evaluate(xtrain,ytrain)

In [ ]:
!pip install -U keras-tuner 
!pip install --upgrade tensorflow jax jaxlib 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("C:/Users/gheva/OneDrive/Desktop/Alphabets_data.csv")
df

In [ ]:
df.isna().sum()

In [ ]:
sc = StandardScaler() # don't perform standardization on target column
x = df.drop(columns=['letter'])# exclude Exited column
y = df['letter']

In [ ]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [ ]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics =['accuracy'])

ann.fit(xtrain,ytrain,epochs = 100)

In [ ]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

In [ ]:
print(classification_report(ytest, ypred))

In [ ]:
ann.evaluate(xtrain,ytrain)

##Hyperparameter tuning##

In [ ]:
list(range(8,128,8))

In [ ]:
def hyper(hp):
    model = Sequential()

    # Add input layer
    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=13))

    # Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))

    # Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [ ]:
tuner.search(xtrain,ytrain,epochs = 3, validation_data = (xtest,ytest))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

In [ ]:
tuned_model.evaluate(xtrain,ytrain)

In [ ]:
tuned_model.evaluate(xtest,ytest)